In [10]:
!unzip Dimitri.zip -d data/Dimitri

Archive:  Dimitri.zip
  inflating: data/Dimitri/PXL_20251116_075638277.jpg  
  inflating: data/Dimitri/PXL_20251116_075644657.jpg  
  inflating: data/Dimitri/PXL_20251116_075634346.jpg  
  inflating: data/Dimitri/PXL_20251116_075636499.jpg  
  inflating: data/Dimitri/PXL_20251116_075634764.jpg  
  inflating: data/Dimitri/PXL_20251116_075643935.jpg  
  inflating: data/Dimitri/PXL_20251116_075642045.jpg  
  inflating: data/Dimitri/PXL_20251116_075637463.jpg  
  inflating: data/Dimitri/PXL_20251116_075641192.jpg  
  inflating: data/Dimitri/PXL_20251116_075640372.jpg  
  inflating: data/Dimitri/PXL_20251116_075643229.jpg  
  inflating: data/Dimitri/PXL_20251116_075635231.jpg  
  inflating: data/Dimitri/PXL_20251116_075641665.jpg  
  inflating: data/Dimitri/PXL_20251116_075633195.jpg  
  inflating: data/Dimitri/PXL_20251116_075635622.jpg  
  inflating: data/Dimitri/PXL_20251116_075637865.jpg  
  inflating: data/Dimitri/PXL_20251116_075640891.jpg  
  inflating: data/Dimitri/PXL_20251116_0756

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import class_weight

np.random.seed(42)
tf.random.set_seed(42)

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 8
EPOCHS = 100
DATA_DIR = './data'


def create_cnn_model(num_classes=3):
    """
    Create a CNN model for face classification trained from scratch.

    Architecture:
    - 3 convolutional blocks with batch normalization and dropout
    - Global average pooling layer
    - 1 dense layer with dropout
    - Softmax output layer

    Args:
        num_classes: Number of output classes

    Returns:
        Compiled Keras Sequential model
    """
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),

        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),

        layers.GlobalAveragePooling2D(),

        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),

        layers.Dense(num_classes, activation='softmax')
    ])

    return model


def prepare_data(data_dir):
    """
    Prepare training and validation data generators with augmentation.

    Applies moderate data augmentation to training set including rotation,
    shifts, flips, and zoom. Validation set only receives normalization.

    Args:
        data_dir: Path to directory containing class subdirectories

    Returns:
        Tuple of (train_generator, val_generator)
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,
        zoom_range=0.15,
        fill_mode='nearest',
        validation_split=0.2
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_generator = val_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_generator, val_generator


def plot_training_history(history, save_path='training_history.png'):
    """
    Plot and save training and validation accuracy and loss curves.

    Args:
        history: Keras History object from model.fit()
        save_path: Path to save the plot image
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(history.history['loss'], label='Train Loss')
    axes[1].plot(history.history['val_loss'], label='Val Loss')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Training history plot saved to {save_path}")
    plt.close()


def evaluate_model(model, val_generator):
    """
    Evaluate model performance and generate confusion matrix.

    Prints classification report with precision, recall, and F1-scores.
    Saves confusion matrix visualization.

    Args:
        model: Trained Keras model
        val_generator: Validation data generator

    Returns:
        Tuple of (predictions, true_labels)
    """
    val_generator.reset()
    predictions = model.predict(val_generator, steps=len(val_generator))
    y_pred = np.argmax(predictions, axis=1)
    y_true = val_generator.classes

    class_names = list(val_generator.class_indices.keys())

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    print("Confusion matrix saved to confusion_matrix.png")
    plt.close()

    return y_pred, y_true


def main():
    """
    Main training pipeline for face classification CNN.

    Pipeline:
    1. Load and prepare data with augmentation
    2. Create CNN model
    3. Compile model with Adam optimizer
    4. Train with callbacks (early stopping, learning rate reduction)
    5. Evaluate and save model
    6. Generate visualizations and metrics
    """
    print("=" * 60)
    print("Face Classification CNN - Training From Scratch")
    print("=" * 60)

    if not os.path.exists(DATA_DIR):
        print(f"\nError: Data directory '{DATA_DIR}' not found!")
        print("Please update the DATA_DIR variable with your data path.")
        return

    print(f"\nLoading data from: {DATA_DIR}")
    train_generator, val_generator = prepare_data(DATA_DIR)

    num_classes = len(train_generator.class_indices)
    print(f"\nNumber of classes: {num_classes}")
    print(f"Class names: {list(train_generator.class_indices.keys())}")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")

    print("\nCreating custom CNN model...")
    model = create_cnn_model(num_classes)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Summary:")
    model.summary()

    class_weights = class_weight.compute_class_weight(
        'balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    class_weight_dict = dict(enumerate(class_weights))

    print(f"\nClass weights (to handle imbalance): {class_weight_dict}")

    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    print("\n" + "=" * 60)
    print("Starting training...")
    print("=" * 60)

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=callbacks,
        class_weight=class_weight_dict,
        verbose=1
    )

    plot_training_history(history)

    print("\n" + "=" * 60)
    print("Evaluating model...")
    print("=" * 60)

    evaluate_model(model, val_generator)

    model.save('face_classifier_final.keras')
    print("\nFinal model saved to: face_classifier_final.keras")

    print("\n" + "=" * 60)
    print("Training Complete!")
    print("=" * 60)
    print("\nGenerated files:")
    print("1. best_model.keras - Best model during training")
    print("2. face_classifier_final.keras - Final trained model")
    print("3. training_history.png - Training curves")
    print("4. confusion_matrix.png - Confusion matrix")


if __name__ == "__main__":
    main()

Face Classification CNN - Training From Scratch

Loading data from: ./data
Found 100 images belonging to 3 classes.
Found 25 images belonging to 3 classes.

Number of classes: 3
Class names: ['Alexis', 'Dimitri', 'Pallav']
Training samples: 100
Validation samples: 25

Creating custom CNN model...

Model Summary:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,555 (435.76 KB)

...


Class weights (to handle imbalance): {0: np.float64(0.8333333333333334), 1: np.float64(1.6666666666666667), 2: np.float64(0.8333333333333334)}

Starting training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 606ms/step - accuracy: 0.8560 - loss: 0.3437
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to best_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 775ms/step - accuracy: 0.8627 - loss: 0.3320 - val_accuracy: 0.2000 - val_loss: 1.1250 - learning_rate: 0.0010
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.9805 - loss: 0.0756
Epoch 2: val_accuracy improved from 0.20000 to 0.44000, saving model to best_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 243ms/step - accuracy: 0.9797 - loss: 0.0758 - val_accuracy: 0.4400 - val_loss: 1.1416 - learning_rate: 0.0010
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.9157 - loss: 0.2660
Epoch 3: val_accuracy did not improve from 0.44000
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 249ms/step - accuracy: 0.9189 - loss: 0.2577 - val_accuracy: 0.4000 - val_loss: 1.2524 - learning_rate: 0.0010
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.9747 - loss: 0.0963
Ep

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix saved to confusion_matrix.png

Final model saved to: face_classifier_final.keras

Training Complete!

Generated files:
1. best_model.keras - Best model during training
2. face_classifier_final.keras - Final trained model
3. training_history.png - Training curves
4. confusion_matrix.png - Confusion matrix


In [16]:
!unzip Pallav.zip -d data/

Archive:  Pallav.zip
   creating: data/Pallav/
  inflating: data/Pallav/Pallav_0000_20251117_023337_979091.jpg  
  inflating: data/Pallav/Pallav_0001_20251117_023338_289954.jpg  
  inflating: data/Pallav/Pallav_0002_20251117_023338_619188.jpg  
  inflating: data/Pallav/Pallav_0003_20251117_023338_946614.jpg  
  inflating: data/Pallav/Pallav_0004_20251117_023339_265542.jpg  
  inflating: data/Pallav/Pallav_0005_20251117_023339_588722.jpg  
  inflating: data/Pallav/Pallav_0006_20251117_023339_905977.jpg  
  inflating: data/Pallav/Pallav_0007_20251117_023340_231977.jpg  
  inflating: data/Pallav/Pallav_0008_20251117_023340_549708.jpg  
  inflating: data/Pallav/Pallav_0009_20251117_023340_877651.jpg  
  inflating: data/Pallav/Pallav_0010_20251117_023341_200514.jpg  
  inflating: data/Pallav/Pallav_0011_20251117_023341_525692.jpg  
  inflating: data/Pallav/Pallav_0012_20251117_023341_881212.jpg  
  inflating: data/Pallav/Pallav_0013_20251117_023342_205256.jpg  
  inflating: data/Pallav/Pall

In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils import class_weight

np.random.seed(42)
tf.random.set_seed(42)

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 8
EPOCHS = 100
DATA_DIR = './data'


def create_transfer_learning_model(num_classes=3):
    """
    Create a transfer learning model using MobileNetV2 pre-trained on ImageNet.

    Uses pre-trained MobileNetV2 as feature extractor with custom classification head.
    The base model starts frozen and can be fine-tuned later.

    Args:
        num_classes: Number of output classes

    Returns:
        Tuple of (model, base_model)
    """
    base_model = keras.applications.MobileNetV2(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model, base_model


def prepare_data(data_dir):
    """
    Prepare training and validation data generators with augmentation.

    Applies moderate data augmentation to training set. Validation set
    only receives normalization.

    Args:
        data_dir: Path to directory containing class subdirectories

    Returns:
        Tuple of (train_generator, val_generator)
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        fill_mode='nearest',
        validation_split=0.2
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_generator = val_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_generator, val_generator


def plot_training_history(history, save_path='training_history_transfer.png'):
    """
    Plot and save training and validation accuracy and loss curves.

    Args:
        history: Keras History object from model.fit()
        save_path: Path to save the plot image
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(history.history['loss'], label='Train Loss')
    axes[1].plot(history.history['val_loss'], label='Val Loss')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Training history plot saved to {save_path}")
    plt.close()


def evaluate_model(model, val_generator):
    """
    Evaluate model performance and generate confusion matrix.

    Prints classification report with precision, recall, and F1-scores.
    Saves confusion matrix visualization.

    Args:
        model: Trained Keras model
        val_generator: Validation data generator

    Returns:
        Tuple of (predictions, true_labels)
    """
    val_generator.reset()
    predictions = model.predict(val_generator, steps=len(val_generator))
    y_pred = np.argmax(predictions, axis=1)
    y_true = val_generator.classes

    class_names = list(val_generator.class_indices.keys())

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix_transfer.png', dpi=300, bbox_inches='tight')
    print("Confusion matrix saved to confusion_matrix_transfer.png")
    plt.close()

    return y_pred, y_true


def main():
    """
    Main training pipeline for face classification using transfer learning.

    Pipeline:
    1. Load and prepare data with augmentation
    2. Create transfer learning model with frozen base
    3. Train classification head
    4. Unfreeze and fine-tune entire model
    5. Evaluate and save model
    6. Generate visualizations and metrics
    """
    print("=" * 60)
    print("Face Classification - Transfer Learning")
    print("=" * 60)

    if not os.path.exists(DATA_DIR):
        print(f"\nError: Data directory '{DATA_DIR}' not found!")
        print("Please update the DATA_DIR variable with your data path.")
        return

    print(f"\nLoading data from: {DATA_DIR}")
    train_generator, val_generator = prepare_data(DATA_DIR)

    num_classes = len(train_generator.class_indices)
    print(f"\nNumber of classes: {num_classes}")
    print(f"Class names: {list(train_generator.class_indices.keys())}")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")

    train_class_counts = np.bincount(train_generator.classes)
    print(f"Training class counts: {dict(zip(train_generator.class_indices.keys(), train_class_counts))}")

    max_count = np.max(train_class_counts)
    class_weights = max_count / train_class_counts
    class_weight_dict = dict(enumerate(class_weights))

    print(f"Class weights: {class_weight_dict}")

    print("\nCreating transfer learning model with MobileNetV2...")
    model, base_model = create_transfer_learning_model(num_classes)

    print("\n" + "=" * 60)
    print("PHASE 1: Training classification head (base frozen)")
    print("=" * 60)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Summary:")
    model.summary()

    callbacks_phase1 = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model_transfer_phase1.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    print("\nTraining classification head...")
    history_phase1 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=30,
        callbacks=callbacks_phase1,
        class_weight=class_weight_dict,
        verbose=1
    )

    print("\n" + "=" * 60)
    print("PHASE 2: Fine-tuning (unfreezing base model)")
    print("=" * 60)

    base_model.trainable = True

    fine_tune_at = len(base_model.layers) - 30
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print(f"\nUnfrozen layers: {sum([1 for layer in model.layers if layer.trainable])}")
    print(f"Frozen layers: {sum([1 for layer in model.layers if not layer.trainable])}")

    callbacks_phase2 = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model_transfer.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    print("\nFine-tuning entire model...")
    history_phase2 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=callbacks_phase2,
        class_weight=class_weight_dict,
        verbose=1
    )

    for key in history_phase1.history.keys():
        history_phase1.history[key].extend(history_phase2.history[key])

    plot_training_history(history_phase1)

    print("\n" + "=" * 60)
    print("Evaluating model...")
    print("=" * 60)

    evaluate_model(model, val_generator)

    model.save('face_classifier_transfer_final.keras')
    print("\nFinal model saved to: face_classifier_transfer_final.keras")

    print("\n" + "=" * 60)
    print("Training Complete!")
    print("=" * 60)
    print("\nGenerated files:")
    print("1. best_model_transfer.keras - Best model during training")
    print("2. face_classifier_transfer_final.keras - Final trained model")
    print("3. training_history_transfer.png - Training curves")
    print("4. confusion_matrix_transfer.png - Confusion matrix")


if __name__ == "__main__":
    main()

Face Classification - Transfer Learning

Loading data from: ./data
Found 100 images belonging to 3 classes.
Found 25 images belonging to 3 classes.

Number of classes: 3
Class names: ['Alexis', 'Dimitri', 'Pallav']
Training samples: 100
Validation samples: 25
Training class counts: {'Alexis': np.int64(40), 'Dimitri': np.int64(20), 'Pallav': np.int64(40)}
Class weights: {0: np.float64(1.0), 1: np.float64(2.0), 2: np.float64(1.0)}

Creating transfer learning model with MobileNetV2...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

PHASE 1: Training classification head (base frozen)

Model Summary:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,620,739 (10.00 MB)

 Trainable params: 361,987 (1.38 MB)

 Non-trainable params: 2,258,752 (8.62 MB)


Training classification head...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6544 - loss: 0.9411
Epoch 1: val_accuracy improved from -inf to 0.96000, saving model to best_model_transfer_phase1.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.6634 - loss: 0.9171 - val_accuracy: 0.9600 - val_loss: 0.3691 - learning_rate: 0.0010
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.9644 - loss: 0.1428
Epoch 2: val_accuracy did not improve from 0.96000
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step - accuracy: 0.9627 - loss: 0.1459 - val_accuracy: 0.9600 - val_loss: 0.2282 - learning_rate: 0.0010
Epoch 3/30
12/13 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 0.9292 - loss: 0.1557
Epoch 3: val_accuracy did not improve from 0.96000
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 264ms/step - accuracy: 0.9322 - loss: 0.1579 - val_accuracy: 0.9600 - val_loss: 0.1389 - learning_rate: 0.0010
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.9792 - loss: 0.0700
Epoch 4: val_accuracy improved fro